In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad 

In [2]:
for i in range(151507,151511):
    print(i)

    input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

    adata_temp = sc.read_visium(input_dir)
    adata_temp.var_names_make_unique()
    adata_temp.obs_names_make_unique()
    adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
    adata_temp.obs['batch'] = adata_label['batch'].astype("category")
    adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
    SVGs = pd.read_csv(input_dir + 'SPARKX.csv', index_col=0)
    adata_temp.uns['SVGs_'+str(i)] = SVGs
    print(adata_temp)
    adata_list.append(adata_temp)

D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
             combinedPval  adjustedPval
MIR1302-2HG      0.481584           1.0
AL627309.1       0.929944           1.0
AL669831.5       0.558528           1.0
FAM87B           0.305351           1.0
LINC00115        0.098451           1.0


In [4]:
SVGs = SVGs.sort_values(by="adjustedPval",ascending=True)
SVGs

,combinedPval,adjustedPval
MBP,5.726466e-284,1.276208e-278
SCGB2A2,2.988574e-253,3.330188e-248
SCGB1D2,1.916310e-179,1.423571e-174
PLP1,5.727676e-167,3.191194e-162
MT3,2.474007e-129,1.102721e-124
...,...,...
C15orf59-AS1,8.331385e-02,1.000000e+00
LOXL1,6.320140e-01,1.000000e+00
PML,1.190956e-01,1.000000e+00
AC079322.1,4.106111e-01,1.000000e+00


In [7]:
sc.pp.neighbors(adata_temp, use_rep='X')

In [12]:
num = [1,5,10,20,50,100,200,500,1000]
for SVGnum in num:
    print(SVGnum)
    print( np.mean( 
        sc.metrics.morans_i(adata_temp, vals = adata_temp[:, SVGs.index.tolist()[:SVGnum]].X.toarray().T)
        ) )
    print( np.mean( 
        sc.metrics.gearys_c(adata_temp, vals = adata_temp[:, SVGs.index.tolist()[:SVGnum]].X.toarray().T)
        ) )


1
0.6427118313661013
0.23487762390365127
2
0.5608862933657133
0.30218406381748597
5
0.5047758070938947
0.34700083645417124
10
0.4527001537455592
0.40704098552085854
20
0.4074597557541263
0.45870133943743563
50
0.36060198238992436
0.513665937451784
100
0.31383972838330976
0.5689229244395393
200
0.2574012418708904
0.6235568466108484
500
0.21073323691109516
0.6695815389907682
1000
0.15564522017442625
0.7230375881693742


In [26]:
print( adata_temp )
print( np.mean( sc.metrics.morans_i(adata_temp, obsm='X_pca') ) )

AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial', 'neighbors'
    obsm: 'spatial', 'X_pca'
    obsp: 'distances', 'connectivities'
0.2476710702106601
